In [ ]:
import os
from unstructured.partition.auto import partition

In [ ]:
def extract_pdf_contents(pdf_file):
    elements = partition(filename=pdf_file)
    text = ''
    for element in elements:
        try:
            text = text + str(element) + '\n'
        except Exception as e:
            print(e)
    return text

In [ ]:
folder = r"2024"
out_folder = r"Summary"

for file in os.listdir(folder):    
    filename = os.path.join(folder, file)
    print(file)
    try:
        text = extract_pdf_contents(filename)
        out_file = os.path.join(out_folder, file.replace("pdf", "txt"))
        with open(out_file, "w") as f:
            f.write(text)

    except Exception as e:
        print(e)        
    

In [ ]:
yoga = r"translation.pdf"
elements = partition(filename=yoga)

In [ ]:
index = 0
for element in elements:    
    print(element)
    index += 1    

In [ ]:
from docx import Document
from lxml import etree
import zipfile
ooXMLns = {'w':'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
#Function to extract all the comments of document(Same as accepted answer)
#Returns a dictionary with comment id as key and comment string as value
def get_document_comments(docxFileName):
    comments_dict={}
    docxZip = zipfile.ZipFile(docxFileName)
    commentsXML = docxZip.read('word/comments.xml')
    et = etree.XML(commentsXML)
    comments = et.xpath('//w:comment',namespaces=ooXMLns)
    for c in comments:
        comment=c.xpath('string(.)',namespaces=ooXMLns)
        comment_id=c.xpath('@w:id',namespaces=ooXMLns)[0]
        comments_dict[comment_id]=comment
    return comments_dict
#Function to fetch all the comments in a paragraph
def paragraph_comments(paragraph,comments_dict):
    comments=[]
    for run in paragraph.runs:
        comment_reference=run._r.xpath("./w:commentReference")
        if comment_reference:
            comment_id=comment_reference[0].xpath('@w:id',namespaces=ooXMLns)[0]
            comment=comments_dict[comment_id]
            comments.append(comment)
    return comments
#Function to fetch all comments with their referenced paragraph
#This will return list like this [{'Paragraph text': [comment 1,comment 2]}]
def comments_with_reference_paragraph(docxFileName):
    document = Document(docxFileName)
    comments_dict=get_document_comments(docxFileName)
    comments_with_their_reference_paragraph=[]
    for paragraph in document.paragraphs:  
        if comments_dict: 
            comments=paragraph_comments(paragraph,comments_dict)  
            if comments:
                comments_with_their_reference_paragraph.append({paragraph.text: comments})
    return comments_with_their_reference_paragraph

In [ ]:
document = "hello.pdf"  #filepath for the input document
print(comments_with_reference_paragraph(document))

In [ ]:
responses = [
    {
        "name": "Alice",
        "like_to_spend_time_with": ["Bob", "Charlie", "David"],
        "not_like_to_spend_time_with": ["Eve", "Frank", "Grace"],
        "selected_by": ["Bob", "Charlie"],
        "think_rejected_by": ["Eve", "Frank"],
    },
    # Add more individuals' responses here...
]


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create an empty graph
G = nx.Graph()

# Add nodes (individuals)
for person in responses:
    G.add_node(person["name"])

# Add edges based on preferences
for person in responses:
    for liked_person in person["like_to_spend_time_with"]:
        G.add_edge(person["name"], liked_person)

# Visualize the graph
pos = nx.spring_layout(G, seed=42)  # Layout algorithm
nx.draw(G, pos, with_labels=True, node_size=800, font_size=10, font_color="black")
plt.title("Sociometric Graph: Preferences for Spending Free Time")
plt.show()
